# 入门云原生AI - 1. 从mnist开始体验

在这个示例中，我们将演示：

* 下载并准备数据
* 利用Arena提交单机训练任务,并且查看训练任务状态和日志，并通过TensorBoard查看训练任务
* 为您的训练结果部署一个模型预测的在线服务。
* 在Notebook中通过Http调用部署的在线预测服务，验证模型准确率。

1.下载TensorFlow样例源代码到 /root/models 目录

In [ ]:
! git clone https://code.aliyun.com/xiaozhou/tensorflow-sample-code.git /root/models/tensorflow-sample-code

2.下载mnist数据到 /root/dataset/mnist

In [ ]:
! mkdir -p /root/dataset/mnist && \
  cd /root/dataset/mnist && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-labels-idx1-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-labels-idx1-ubyte.gz

3.创建训练结果${HOME}/output

In [5]:
! mkdir -p /root/output

4.查看目录结构, 其中`dataset`是数据目录，`models`是模型代码目录，`output`是训练结果目录。

In [ ]:
! tree -I ai-starter -L 3 /root

5.检查可用GPU资源

In [ ]:
! arena top node

6.通过Arena提交训练任务

6.1 可以根据您的需要设置JOB_NAME，建议在多人共同使用的时候，设置自己独有的JOB_NAME

6.2 这里 `$USER_DATA_NAME` 代表您的Notebook 使用的共享存储，存储的根目录和Notebook中`/root`对应。 由集群管理员在部署工作环境时指定[部署数据科学家工作环境（Notebook）](../docs/setup/SETUP_NOTEBOOK.md)创建。 
* `$USER_DATA_NAME` 是存放您私有数据的共享存储，文件内容对应Notebook的 `/root` 目录
* `$PUBLIC_DATA_NAME` 是存放公共数据的共享存储，文件内容对应Notebook的 `/root/public` 目录

在下面的提交命令中：
* `--data=$USER_DATA_NAME:/training` 表示将共享存储映射到训练任务的`/training`目录。
* `/training`目录下的子目录`/training/models/tensorflow-sample-code` 就是步骤1拷贝源代码的位置
* `/training`目录下的子目录`/training/dataset/mnist`就是步骤2下载数据的位置
* `/training`目录下的子目录`/training/output`就是步骤3创建的训练结果输出的位置。

In [ ]:
# Set the Job Name
%env JOB_NAME=tf-mnist
# Submit a training job 
# using code and data from Data Volume
!arena submit tf \
             --name=$JOB_NAME \
             --gpus=1 \
             --data=$USER_DATA_NAME:/training \
             --tensorboard \
             --image=tensorflow/tensorflow:1.11.0-gpu-py3 \
             --logdir=/training/output/mnist \
             "python /training/models/tensorflow-sample-code/tfjob/docker/mnist/main.py --max_steps 10000 --data_dir /training/dataset/mnist --log_dir /training/output/mnist"

> - `Arena`命令的`--logdir`指定`tensorboard`从训练任务的指定目录读取event  
> - 完整参数可以参考[命令行文档](https://github.com/kubeflow/arena/blob/master/docs/cli/arena_submit_tfjob.md)

7.检查模型训练状态，当任务状态从`Pending`转为`Running`后就可以查看日志和GPU使用率了。这里`-e`为了方便检查任务`Pending`的原因。通常看到`[Pulling] pulling image "tensorflow/tensorflow:1.11.0-gpu-py3"`代表容器镜像过大，导致任务处于`Pending`。这时可以重复执行下列命令直到任务状态变为`Running`。

In [ ]:
! arena get $JOB_NAME -e 2>&1

8.实时检查日志，此时可以通过调整`--tail=`的数值展示输出的行数。默认为显示全部日志。

In [ ]:
! arena logs --tail=50 $JOB_NAME

9.查看实时训练的GPU使用情况

In [ ]:
! arena top job $JOB_NAME

10.通过TensorBoard查看训练趋势。您可以使用 `192.168.1.117:30670` 访问 Tensorboard。如果您通过笔记本电脑无法直接访问 Tensorboard，可以考虑在您的笔记本电脑使用 `sshuttle`。例如：`sshuttle -r root@41.82.59.51 192.168.0.0/16`。其中`41.82.59.51`为集群内某个节点的外网IP，且该外网IP可以通过ssh访问。

In [ ]:
# show job detail
! arena get $JOB_NAME

![](1-1-tensorboard.jpg)

11.查看模型训练产生的结果, 在`output`下生成了训练结果.

`/training/output/mnist` 目录中是训练过程中产生的checkpoint文件，代表训练结束时模型的变量状态。

In [ ]:
! tree -I ai-starter -L 3 ${HOME}

12.将训练过程中产生的checkpoint转换为模型文件.

checkpoint文件不能直接用于部署预测服务，需要进行一次模型导出，将checkpoint文件中的模型状态导出为可以被预测服务识别的模型文件。 我们可以通过arena 提交一个转换任务，声明将`output/mnist`目录下的checkpoint文件导出到`output/mnist-model`目录。

In [ ]:
! arena submit tf \
     --name=export-model \
     --workers=1 \
     --data=$VOLUME_NAME:/training \
     --image=tensorflow/tensorflow:1.11.0-gpu-py3 \
     "python /training/models/tensorflow-sample-code/tfjob/docker/export-model/export_model.py \
     --checkpoint_path=/training/output/mnist /training/output/mnist-model/ "

12.1.查看模型导出任务的执行状态

In [ ]:
! arena get export-model

12.2导出任务执行完毕后，可以在`output/mnist-model`目录中看到导出的模型文件。

In [ ]:
! tree -I ai-starter -L 3 output/mnist-model

13.部署预测服务

我们得到可以用于部署预测服务的模型文件后，可以通过`arena serve` 提交一个模型预测的在线任务。 

* `--data=$USER_DATA_NAME:/training` 表示将共享存储目录挂载到预测服务的`/training`目录
* `--modelPath=/training/output/mnist-model` 表示预测服务使用的模型文件目录，就是我们在步骤12中导出的模型文件位置

In [ ]:
! arena serve tensorflow \
    --servingName=mnist \
    --modelName=mnist \
    --image=tensorflow/serving:latest  \
    --data=$USER_DATA_NAME:/training \
    --modelPath=/training/output/mnist-model

13.2查看预测服务

我们可以查到到预测服务的部署状态，以及入口访问地址。

In [ ]:
! arena serve list

14.验证预测服务

14.1. 定义一个python方法，这个方法会从mnist的测试数据集中，随机选择一张图片，作为请求的参数，通过http方式调用预测服务。执行完成后会同时打印图片的真实值，和通过预测服务推理得到的值。 您可以用于判断模型的准确率是否满足要求。

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import requests
import json
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
data_dir="/root/dataset/mnist/"
mnist = input_data.read_data_sets(data_dir, one_hot=True)
test_images = mnist.test.images
test_labels = mnist.test.labels
digits = ['0','1','2','3','4','5','6','7','8','9']
# -*- coding: utf-8 -*-
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

def predict(url, num):
    test_cls = np.argmax(test_labels, axis=1)
    show(num, 'The Picture is {}'.format(test_cls[num]))
    headers = {"content-type": "application/json"}
    metadata=requests.get(url + '/metadata')
    data = json.dumps({"signature_name": "predict_images", "inputs": test_images[num].reshape(1, 784).tolist()})
    json_response = requests.post(model_api+':predict', data=data, headers=headers)
    scores = json.loads(json_response.text)['outputs']
    predicted_digits_idx = np.argmax(scores)
    print('预测结果是: {}'.format(digits[predicted_digits_idx]))
    return scores

def random_image_predict(model_api):
    random_image = random.randint(0,len(test_images)-1)
    score = predict(model_api, random_image)

14.2调用函数。
我们可以得到真实值和预测值。 您可以多执行几次或者修改代码增加执行次数，比较预测结果和真是结果，判断模型的预测准确率。

这里`mnist-tensorflow-serving` 代表预测服务的服务域名，您也可以改为IP地址，IP地址的获取在12步中，通过`arena serve list` 可以得到的预测服务的服务IP。

In [ ]:
model_api='http://mnist-tensorflow-serving:8501/v1/models/mnist'
random_image_predict(model_api)

15.删除已经完成的任务

In [ ]:
# delete job
! arena delete $JOB_NAME
! arena delete export-model
# delete serving job
! arena serve delete mnist

恭喜！您已经使用 `arena` 成功运行了训练作业，而且还能轻松检查 Tensorboard。

总结，希望您通过本次演示了解：
1. 如何准备代码和数据，并将其放入数据卷中
2. 如何在训练任务中引用数据卷，并且使用其中的代码和数据
3. 如何利用arena管理您的训练任务。
4. 为您的训练结果部署一个模型预测的在线服务。
5. 在Notebook中通过Http调用部署的在线预测服务，验证模型准确率。

以上是使用`Arena`在云上进行模型训练的例子，您可以通过修改代码`/root/models/tensorflow-sample-code/tfjob/docker/mnist/main.py`重新提交，实现迭代的模型开发目的。